# Install dependencies

In [2]:
#!conda install Faker openai aiohttp pyarrow pandas

# Generate fake reviews

Performs poorly because it doesn't have fine-tuning


In [3]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed


def generate_and_save_review(executor, i):
    chat_completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user",
                   "content": "Write the text for a review for a food item. Do not use placeholders, fill in details with whatever food, business, restaurant, franchise, or whatever you decide, fill in gaps with some food that makes sense in the context of the review. Length of the review should vary from 100-500 words. Your review does not have to be positive, use whatever tone and emotion you feel like, just keep the overall tone of the review consistent. Try to avoid using a cookie cutter template, make the review unique and interesting."}]
    )
    text = chat_completion.choices[0].message.content.strip()
    print(text)
    # Execute DB insert in the thread pool
    future_db = executor.submit(conn.execute, "INSERT INTO FakeFoodReviewSourceText (Text) VALUES (?)", (text,))
    future_db.result()  # Wait for DB operation to complete
    conn.commit()

    # File writing
    filename = f"./amazon_food_reviews_dataset/fake-food-reviews/review_{time.time()}.txt"
    with open(filename, "w") as f:
        f.write(text)


with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(generate_and_save_review, executor, i) for i in range(1000)]

    # Optionally, wait for all futures to complete (this is implicit when exiting the context manager)
    for future in as_completed(futures):
        future.result()  # This would raise exceptions if any occurred within the threads
    

NameError: name 'openai_client' is not defined

In [4]:
# Create in memory kms for parquet encryption

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.
import base64

import pyarrow.parquet.encryption as pe


class InMemoryKmsClient(pe.KmsClient):
    """This is a mock class implementation of KmsClient, built for testing
    only.
    """

    def __init__(self, config):
        """Create an InMemoryKmsClient instance."""
        pe.KmsClient.__init__(self)
        self.master_keys_map = config.custom_kms_conf

    def wrap_key(self, key_bytes, master_key_identifier):
        """Not a secure cipher - the wrapped key
        is just the master key concatenated with key bytes"""
        master_key_bytes = self.master_keys_map[master_key_identifier].encode(
            'utf-8')
        wrapped_key = b"".join([master_key_bytes, key_bytes])
        result = base64.b64encode(wrapped_key)
        return result

    def unwrap_key(self, wrapped_key, master_key_identifier):
        """Not a secure cipher - just extract the key from
        the wrapped key"""
        expected_master_key = self.master_keys_map[master_key_identifier]
        decoded_wrapped_key = base64.b64decode(wrapped_key)
        master_key_bytes = decoded_wrapped_key[:16]
        decrypted_key = decoded_wrapped_key[16:]
        if (expected_master_key == master_key_bytes.decode('utf-8')):
            return decrypted_key
        raise ValueError("Incorrect master key used",
                         master_key_bytes, decrypted_key)


def verify_file_encrypted(path):
    """Verify that the file is encrypted by looking at its first 4 bytes.
    If it's the magic string PARE
    then this is a parquet with encrypted footer."""
    with open(path, "rb") as file:
        magic_str = file.read(4)
        # Verify magic string for parquet with encrypted footer is PARE
        assert magic_str == b'PARE'


# Copy sqlite database to parquet

## Example code from
https://github.com/apache/arrow/blob/main/python/examples/dataset/write_dataset_encrypted.py#L25
https://github.com/apache/arrow/blob/45918a90a6ca1cf3fd67c256a7d6a240249e555a/python/pyarrow/tests/parquet/test_encryption.py#L68-L98

In [10]:
from pyarrow._parquet_encryption import CryptoFactory, timedelta
import sqlite3
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.parquet.encryption as pe
PARQUET_NAME = 'encrypted_table.in_mem.parquet'
FOOTER_KEY = b"0123456789112345"
FOOTER_KEY_NAME = "footer_key"
COL_KEY = b"1234567890123450"
COL_KEY_NAME = "col_key"

# Connect to the SQLite database
conn = sqlite3.connect('./large_db.sqlite')  # Replace 'your_database.db' with your database file

# Retrieve the data into a Pandas DataFrame
query = "SELECT * FROM Reviews LIMIT 10"  # Replace 'your_table' with your actual table name
df = pd.read_sql_query(query, conn)

# Convert the DataFrame to a PyArrow Table
table = pa.Table.from_pandas(df)
# Export the PyArrow Table to Parquet format
encryption_config = pe.EncryptionConfiguration(
    column_keys={COL_KEY_NAME: ["PrivateKey", "Text"]},
    footer_key=FOOTER_KEY_NAME,
    encryption_algorithm="AES_GCM_V1",
    cache_lifetime=timedelta(minutes=5),
    # data_key_length_bits=256,
)
kms_connection_config = pe.KmsConnectionConfig(
    custom_kms_conf={
        FOOTER_KEY_NAME: FOOTER_KEY.decode("UTF-8"),
        COL_KEY_NAME: COL_KEY.decode("UTF-8"),
    }
)
def kms_factory(kms_connection_configuration):
    return InMemoryKmsClient(kms_connection_configuration)

print(encryption_config)
print(kms_connection_config)
facotyr = CryptoFactory(kms_factory)

pq.write_table(
    table,
    'output_file.parquet',
    ## {‘NONE’, ‘SNAPPY’, ‘GZIP’, ‘BROTLI’, ‘LZ4’, ‘ZSTD’}.
    ## Note can set per column, compression={'n_legs': 'snappy', 'animal': 'gzip'}
    compression='snappy',
    # row_group_size=1000000,
    encryption_properties=facotyr.file_encryption_properties(kms_connection_config, encryption_config),
)
verify_file_encrypted("output_file.parquet")
print("done")
# Close the SQLite connection
conn.close()


done


In [16]:
# Verify contents of parquet file
decryption_properties = facotyr.file_decryption_properties(
    kms_connection_config)
parquet_file = pq.ParquetFile('output_file.parquet', decryption_properties=decryption_properties)
print(parquet_file.metadata)
print(parquet_file.schema)
print(parquet_file.read_row_group(0, columns=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text']))

  created_by: parquet-cpp-arrow version 15.0.0
  num_columns: 12
  num_rows: 10
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 7748
required group field_id=-1 schema {
  optional int64 field_id=-1 Id;
  optional binary field_id=-1 ProductId (String);
  optional binary field_id=-1 UserId (String);
  optional binary field_id=-1 ProfileName (String);
  optional int64 field_id=-1 HelpfulnessNumerator;
  optional int64 field_id=-1 HelpfulnessDenominator;
  optional int64 field_id=-1 Score;
  optional int64 field_id=-1 Time;
  optional binary field_id=-1 Summary (String);
  optional binary field_id=-1 Text (String);
  optional binary field_id=-1 EthAddress (String);
  optional binary field_id=-1 PrivateKey (String);
}

pyarrow.Table
Id: int64
ProductId: string
UserId: string
ProfileName: string
HelpfulnessNumerator: int64
HelpfulnessDenominator: int64
Score: int64
Time: int64
Summary: string
Text: string
----
Id: [[1,2,3,4,5,6,7,8,9,10]]
ProductId: [["B001E4KFG0","B00813GRG4","